In [1]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df = pd.read_csv("spam.csv",encoding="latin")
input_texts = df['v2']
labels = df['v1']

In [3]:
!head edger_allan.txt

'head' is not recognized as an internal or external command,
operable program or batch file.


input_texts = []
labels = []
input_files = [
  'edger-allen.txt',
  'robert-frost.txt',
]
for label,f in enumerate(input_files): #enumerate on list give you index 0 1 so force and also actual val
    for line in open(f):
        line = line.rstrip().lower() #trimming
        if line: # to make sure line is not empty
            line = line.translate(str.maketrans('', '', string.punctuation)) #remove punctiations

            input_texts.append(line)
            labels.append(label)
            

In [4]:
input_texts

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [5]:
labels

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [8]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [9]:
len(Ytrain) 

4179

In [10]:
len(Ytest)

1393

In [11]:
idx =1 
word2idx = {"<unk>":0}
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [12]:
word2idx

{'<unk>': 0,
 '\\YEH': 1,
 'I': 2,
 'AM': 3,
 'DEF': 4,
 'UP4': 5,
 'SOMETHING': 6,
 'SAT': 7,
 'think': 8,
 'asking': 9,
 'for': 10,
 'a': 11,
 'gym': 12,
 'is': 13,
 'the': 14,
 'excuse': 15,
 'lazy': 16,
 'people.': 17,
 'jog.': 18,
 'Dont': 19,
 'put': 20,
 'your': 21,
 'phone': 22,
 'on': 23,
 'silent': 24,
 'mode': 25,
 'ok': 26,
 'You': 27,
 'are': 28,
 'right.': 29,
 'Meanwhile': 30,
 "how's": 31,
 'project': 32,
 'twins': 33,
 'comin': 34,
 'up': 35,
 'Not': 36,
 'yet': 37,
 'chikku..wat': 38,
 'abt': 39,
 'u?': 40,
 'Update_Now': 41,
 '-': 42,
 'Xmas': 43,
 'Offer!': 44,
 'Latest': 45,
 'Motorola,': 46,
 'SonyEricsson': 47,
 '&': 48,
 'Nokia': 49,
 'FREE': 50,
 'Bluetooth!': 51,
 'Double': 52,
 'Mins': 53,
 '1000': 54,
 'Txt': 55,
 'Orange.': 56,
 'Call': 57,
 'MobileUpd8': 58,
 '08000839402': 59,
 'or': 60,
 'call2optout/F4Q=': 61,
 '0A$NETWORKS': 62,
 'allow': 63,
 'companies': 64,
 'to': 65,
 'bill': 66,
 'SMS,': 67,
 'so': 68,
 'they': 69,
 'responsible': 70,
 'their': 71

In [13]:
len(word2idx)

13008

In [14]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)
train_text_int[100:105]

[[949,
  950,
  951,
  461,
  711,
  438,
  68,
  952,
  714,
  115,
  438,
  216,
  143,
  222,
  953,
  351,
  954],
 [955,
  956,
  351,
  957,
  245,
  958,
  21,
  321,
  746,
  959,
  28,
  101,
  35,
  65,
  746,
  2,
  960,
  952,
  85,
  323,
  961,
  554,
  10,
  101,
  962,
  963,
  962,
  964,
  965,
  13,
  961,
  2,
  419,
  263,
  966],
 [2, 113, 967, 126, 968, 969],
 [319, 101, 970, 60, 971, 405, 819, 972, 113, 11, 821, 973],
 [974, 975, 631, 411, 976, 977, 132, 143, 500, 978, 85, 979]]

In [15]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)


In [16]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0) #here we populate a0 and by0 with class 0
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1) #here we populate a1 and by1 with class 1

In [17]:
A0[111]

array([1., 1., 1., ..., 1., 1., 1.])

In [18]:
A0

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 2., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 2., 1.],
       [1., 1., 2., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [19]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [20]:
print(A0[0])

[7.68757688e-05 7.68757688e-05 7.68757688e-05 ... 7.68757688e-05
 7.68757688e-05 7.68757688e-05]


In [21]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)


In [22]:
logA0

array([[-9.47331983, -9.47331983, -9.47331983, ..., -9.47331983,
        -9.47331983, -9.47331983],
       [-9.4733967 , -9.4733967 , -8.78024952, ..., -9.4733967 ,
        -9.4733967 , -9.4733967 ],
       [-9.55194229, -9.55194229, -9.55194229, ..., -9.55194229,
        -9.55194229, -9.55194229],
       ...,
       [-9.4733967 , -9.4733967 , -9.4733967 , ..., -9.4733967 ,
        -8.78024952, -9.4733967 ],
       [-9.4733967 , -9.4733967 , -8.78024952, ..., -9.4733967 ,
        -9.4733967 , -9.4733967 ],
       [-9.4733967 , -9.4733967 , -9.4733967 , ..., -9.4733967 ,
        -9.4733967 , -9.4733967 ]])

In [23]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.8648001914333573, 0.13519980856664274)

In [24]:
# build a classifier
#the class go on each list in text_int and count probabilty in A0,A1,PI0,PI1 and add prior and see which class on top
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):#take as input a line of text repesented as text and class that indicate which markov model to use
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [25]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [26]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.998085666427375


In [27]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.968413496051687


In [28]:
from sklearn.metrics import confusion_matrix, f1_score

In [29]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[3614,    0],
       [   8,  557]], dtype=int64)

In [30]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[1211,    0],
       [  44,  138]], dtype=int64)

In [31]:
f1_score(Ytrain, Ptrain)

0.9928698752228164

In [32]:
f1_score(Ytest, Ptest)

0.8624999999999999